In [2]:
!cypress-kinit

In [28]:
import sys
import os

sys.path.insert(0, '/usr/hdp/2.6.0.3-8/spark2/python')
sys.path.insert(0, '/usr/hdp/2.6.0.3-8/spark2/python/lib/py4j-0.10.4-src.zip')

os.environ['SPARK_HOME'] = '/usr/hdp/2.6.0.3-8/spark2/'
os.environ['SPARK_CONF_DIR'] = '/etc/hadoop/synced_conf/spark2/'
os.environ['PYSPARK_PYTHON'] = '/software/anaconda3/4.2.0/bin/python'

import pyspark
conf = pyspark.SparkConf()
conf.setMaster("yarn")
conf.set("spark.driver.memory","15g")
conf.set("spark.executor.memory","60g")
conf.set("spark.executor.instances","6")
conf.set("spark.executor.cores","12")

sc = pyspark.SparkContext(conf=conf)

In [29]:
conf.set("spark.hadoop.validateOutputSpecs", "false")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
sqlContext = pyspark.SQLContext(sc)
#need to specify input and output
indirectory = "677project/data"
outdirectory = "677project/output/sparkoutput"

In [30]:
readrdd = sc.newAPIHadoopFile(indirectory+"/dump.ch2oh33_prod.lammpstrj",
                              'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
                              'org.apache.hadoop.io.LongWritable',
                              'org.apache.hadoop.io.Text',
                              conf={'textinputformat.record.delimiter': 'ITEM: TIMESTEP'})
readrdd.repartition(100).cache()

MapPartitionsRDD[7] at coalesce at NativeMethodAccessorImpl.java:0

In [31]:
import string
import math
import re
import numpy as np

class Calculation(object):

    rC = [5]                                                                                                           ################
    rHS = [3, 4]                                                                                                            ################
    rOS = [6]
    rHW = [1]
    rOW = [2]

    def __init__(self, a, b, c, cosalpha, cosbeta, cosgamma):
        self.a = a
        self.b = b
        self.c = c
        self.cosalpha = cosalpha
        self.cosbeta = cosbeta
        self.cosgamma = cosgamma

    def init(self, items):
        temp = []
        coords_replica = {}  # the real coords that are used, one of the 9 replica
        x = []
        tmp = []
        OW = []
        h1 = h2 = o1 = o2 = [None] * 100000   # at least 8 times larger than one box h2o atoms.
    
        type34 = items.type3 + items.type4
        h1 = self.findNonperiodic (self.rHS,type34)   # item# and x, y, z for each h1 or h2 or o1 or o2
        h2 = self.find (self.rHW,items.type1)   # item# and x, y, z for each h1 or h2 or o1 or o2
        o1 = self.findNonperiodic (self.rOS,items.type6)   # item# and x, y, z for each h1 or h2 or o1 or o2
        o2 = self.find (self.rOW,items.type2)   # item# and x, y, z for each h1 or h2 or o1 or o2
        h1 = [k for k in h1 if k != None]  #h1
        h2 = [k for k in h2 if k != None]  #h2
        o1 = [k for k in o1 if k != None]  #o1
        o2 = [k for k in o2 if k != None]  #o2
    
        for i in range(len(o1)):
            for j in range(len(o2)):
                do1o2 = self.dist(o1[i] , o2[j])
                if do1o2 <= float(13.5):
                    for l in range(len(h2)):
                        do2h2 = self.dist(o2[j],h2[l])
                        if do2h2<=1.2:
                            do1h2 = self.dist(o1[i] , h2[l])
                            A1 = self.ang( do1o2 , do1h2, do2h2)
                            A2 = self.ang( do2h2 , do1h2, do1o2)
                            #if A2>120 and do1h2<6.25:
                            if A1<=30 and A2>=120 and do1h2<=6.25:
                                for n in range(len(h2)):
                                    if self.dist(o2[j], h2[n])<=1.2:
                                        tmp.append((o2[j][4],h2[n][4]))
                                        if (o2[j][4] not in coords_replica.keys()):
                                            coords_replica[o2[j][4]] = (o2[j][1],o2[j][2],o2[j][3])  # save found O, H coords, x y z
                                        if (h2[n][4] not in coords_replica.keys()):
                                            coords_replica[h2[n][4]] = (h2[n][1],h2[n][2],h2[n][3])  # save found O, H coords, x y z
                    for l in range(len(h1)):
                        do1h1 = self.dist(o1[i],h1[l])
                        if do1h1<=1.2:
                            do2h1 = self.dist(o2[j] , h1[l])
                            # print do1h1,do1o2,do2h1
                            A1 = self.ang( do1o2 , do2h1, do1h1 )
                            A2 = self.ang( do1h1 ,do2h1, do1o2 )
                            #if  A2>120 and do2h1<6.25:
                            if  A1<=30 and A2>=120 and do2h1<=6.25:
                                for m in range(len(h2)):
                                    if self.dist(o2[j],h2[m])<=1.2:  # output HBed Ow and corresponed Hw. At least two pairs, Ow-Hw1 AND Ow-Hw2
                                        tmp.append((o2[j][4],h2[m][4])) # o2[j][4] is j line with item# ([4], already reordered
                                        if (o2[j][4] not in coords_replica.keys()):
                                            coords_replica[o2[j][4]] = (o2[j][1],o2[j][2],o2[j][3])  # save found O, H coords, x y z
                                        if (h2[m][4] not in coords_replica.keys()):
                                            coords_replica[h2[m][4]] = (h2[m][1],h2[m][2],h2[m][3])  # save found O, H coords, x y z
        tmp = list(set(tmp))   # convert 2 lines(2 pairs in 1 h2o mlc: Ow-Hw1 AND Ow-Hw2) to 1 lines: Ow Hw1 Hw2, thus deletes repeating Ow
        for p in range(len(tmp)):
            for q in range(p+1,len(tmp)):  # set the repeated value to -1
                if(tmp[p][0] == tmp[q][0] and tmp[p][1] == tmp[q][1]):
                    tmp[q][0]= -1
            for t in range(p+1, len(tmp)):
                if(tmp[t][0]>0 and tmp[p][0] == tmp[t][0]):
                    OW.append((tmp[p][0],tmp[p][1],tmp[t][1]))
        #print (coords_replica)
        #print(OW)
        #print(len(OW))
        return OW, coords_replica

    def dist(self, X, Y):
        x1 = X[1]
        x2 = Y[1]
        y1 = X[2]
        y2 = Y[2]
        z1 = X[3]
        z2 = Y[3]
        return 2*(x1-x2)*(y1-y2)*self.a*self.b*self.cosgamma + \
               2*(y1-y2)*(z1-z2)*self.b*self.c*self.cosalpha + \
               2*(x1-x2)*(z1-z2)*self.a*self.c*self.cosbeta + \
               (x1-x2)*(x1-x2)*self.a*self.a + \
               (y1-y2)*(y1-y2)*self.b*self.b + (z1-z2)*(z1-z2)*self.c*self.c

    def ang(self,d1,d2,d3):# Note: square of the distances
        return math.degrees(math.acos((d1 + d2 - d3)/(2.0 * math.sqrt(d1) * math.sqrt(d2))))

    def findNonperiodic(self,y,c):
        x = [None] * len(c)
        for i in range(len(c)):
            x[i] = ((c[i].atom_type,c[i].atom_x,c[i].atom_y,c[i].atom_z,c[i].atom_id))  # read the whole line and reorder each string
        return (x)


        #below is periodic boundary #
    def find(self,y,c):
        x=grab= grabxp = grabxn = grabyp = grabyn = grabxpyp = grabxnyn = grabxnyp = grabxpyn = [None] * len(c) * 9
        for i in range(len(c)):
            grab = ((c[i].atom_type,c[i].atom_x,c[i].atom_y,c[i].atom_z,c[i].atom_id))
            grabxp = ((c[i].atom_type,c[i].atom_x+1,c[i].atom_y,c[i].atom_z,c[i].atom_id))
            grabxn = ((c[i].atom_type,c[i].atom_x-1,c[i].atom_y,c[i].atom_z,c[i].atom_id))
            grabyp = ((c[i].atom_type,c[i].atom_x,c[i].atom_y+1,c[i].atom_z,c[i].atom_id))
            grabyn = ((c[i].atom_type,c[i].atom_x,c[i].atom_y-1,c[i].atom_z,c[i].atom_id))
            grabxpyp = ((c[i].atom_type,c[i].atom_x+1,c[i].atom_y+1,c[i].atom_z,c[i].atom_id))
            grabxpyn = ((c[i].atom_type,c[i].atom_x+1,c[i].atom_y-1,c[i].atom_z,c[i].atom_id))
            grabxnyp = ((c[i].atom_type,c[i].atom_x-1,c[i].atom_y+1,c[i].atom_z,c[i].atom_id))
            grabxnyn = ((c[i].atom_type,c[i].atom_x-1,c[i].atom_y-1,c[i].atom_z,c[i].atom_id))
            x[9*i+0]=grab
            x[9*i+1]=grabxp
            x[9*i+2]=grabxn
            x[9*i+3]=grabyp
            x[9*i+4]=grabyn
            x[9*i+5]=grabxpyp
            x[9*i+6]=grabxpyn
            x[9*i+7]=grabxnyp
            x[9*i+8]=grabxnyn
        return (x)

    def coor(self, xlo, xhi, xy, ylo, yhi, xz, zlo, zhi, yz):  # calculate a,b,c,cosalpha,cosbeta,cosgamma
        data = [None] * 6
        xlo_bound = float(xlo)
        xhi_bound = float(xhi)
        xy =  float(xy)
        ylo_bound =  float(ylo)
        yhi_bound =  float(yhi)
        xz =  float(xz)
        zlo =  float(zlo)
        zhi =  float(zhi)
        yz =  float(yz)
        xlo = xlo_bound - min(0.0,xy,xz,xy+xz)
        xhi = xhi_bound - max(0.0,xy,xz,xy+xz)
        ylo = ylo_bound - min(0.0,yz)
        yhi = yhi_bound - max(0.0,yz)
        lx = xhi-xlo
        ly = yhi-ylo
        lz = zhi-zlo
        self.a = lx   #a
        self.b = math.sqrt(ly*ly+xy*xy)   #b
        self.c = math.sqrt(lz*lz+xz*xz+yz*yz)  #c
        self.cosalpha = (xy*xz+ly*yz)/self.b/self.c   #cosalpha
        self.cosbeta = xz/self.c    #cosbeta
        self.cosgamma = xy/self.b   #cosgamma

In [32]:
class Atom(object):

    def __init__(self, atom_id,
                 atom_type, atom_x, atom_y, atom_z):
        self.atom_id = atom_id
        self.atom_type = atom_type
        self.atom_x = atom_x
        self.atom_y = atom_y
        self.atom_z = atom_z

class BoxBounds(object):

    def __init__(self, bounds_xyz1,
                 bounds_xyz2, bounds_xyz3):
        self.bounds_xyz1 = bounds_xyz1
        self.bounds_xyz2 = bounds_xyz2
        self.bounds_xyz3 = bounds_xyz3

class Frame(object):
    
    def __init__(self, timestep, type1, type2, type3,
                 type4, type5, type6, type7):
        self.timestep = timestep
        self.type1 = type1
        self.type2 = type2
        self.type3 = type3
        self.type4 = type4
        self.type5 = type5
        self.type6 = type6
        self.type7 = type7


In [33]:
import os
import sys
from pyspark import SparkContext, SparkConf, SQLContext



def parse_data(data):
    #reading the data in
    #data here is a frame block
    info = data.split("ITEM: NUMBER OF ATOMS")
    #convert to string and replace \n
    timestep = int(''.join(info[0]).replace("\n",""))

    info1 = ''.join(info[1])
    #choose a way to separate the record
    if "xy xz yz" in info1:
        info = info1.split("ITEM: BOX BOUNDS xy xz yz pp pp pp")
    else:
        info = info1.split("ITEM: BOX BOUNDS pp pp pp")
    
    num_of_atoms = int(''.join(info[0].replace("\n","")))
    #box_bounds
    info = ''.join(info[1]).split("ITEM: ATOMS id type xs ys zs")
    box_bounds = ''.join(info[0])
    xyz = box_bounds.split("\n")
    #skip \n
    xyz1 = ''.join(xyz[1]).split()
    #if number of records is  less than 3, add one more record to default 0.0
    if len(xyz1) < 3:
        xyz1.append('0.0')
    xyz2 = ''.join(xyz[2]).split()
    if len(xyz2) < 3:
        xyz2.append('0.0')
    xyz3 = ''.join(xyz[3]).split()
    if len(xyz3) < 3:
        xyz3.append('0.0')
    bounds = BoxBounds((float(xyz1[0]),float(xyz1[1]), float(xyz1[2])),
                       (float(xyz2[0]),float(xyz2[1]), float(xyz2[2])),
                       (float(xyz3[0]), float(xyz3[1]), float(xyz3[2])))
    #atoms
    atoms = ''.join(info[1]).split("\n")
    #atoms_list = []
    type1 = []
    type2 = []
    type3 = []
    type4 = []
    type5 = []
    type6 = []
    type7 = []
    for atom in atoms:
        #skip null elements
        if atom == '':
            continue
        atom_elements = atom.split(" ")
        t_atom =Atom(int(atom_elements[0]), int(atom_elements[1]), float(atom_elements[2]),
                               float(atom_elements[3]), float(atom_elements[4]))
        if(t_atom.atom_type == 7): type7.append(t_atom)
        elif(t_atom.atom_type == 2): type2.append(t_atom)
        elif(t_atom.atom_type == 1): type1.append(t_atom)
        elif(t_atom.atom_type == 3): type3.append(t_atom)
        elif(t_atom.atom_type == 4): type4.append(t_atom)
        elif(t_atom.atom_type == 5): type5.append(t_atom)
        elif(t_atom.atom_type == 6): type6.append(t_atom)

    temp_atoms = Frame(timestep, type1, type2, type3, type4, type5, type6, type7)
    c = Calculation(0.0,0.0,0.0,0.0,0.0,0.0)#init a,b,c,cosalpha,cosbeta,cosgamma
    xlo = bounds.bounds_xyz1[0]
    xhi = bounds.bounds_xyz1[1]
    xy = bounds.bounds_xyz1[2]
    ylo = bounds.bounds_xyz2[0]
    yhi = bounds.bounds_xyz2[1]
    xz = bounds.bounds_xyz2[2]
    zlo = bounds.bounds_xyz3[0]
    zhi = bounds.bounds_xyz3[1]
    yz = bounds.bounds_xyz3[2]
    #do calculation
    c.coor(xlo, xhi, xy, ylo, yhi, xz, zlo, zhi, yz)
    hb_list, real_coords = c.init(temp_atoms)
    hb_length = len(hb_list)
    #print(timestep)
    #print("hb_list:", hb_list)
    #print("len",hb_length)
    #print("real_coords:", real_coords)
    return timestep, hb_length, hb_list, real_coords


In [34]:
%%time
!hdfs dfs -rm -r -skipTrash 677project/output/sparkoutput/dump.ch2oh33_prod.lammpstrj
frame = readrdd.values().filter(lambda x:x!="").map(parse_data).cache()
frame.saveAsTextFile(outdirectory + "/dump.ch2oh33_prod.lammpstrj")

avgHB = frame.map(lambda x: int(x[1])).reduce(lambda x,y:x+y)/frame.count()
#dataframe = sqlContext.createDataFrame(frame,['timestep','hb_length','hb_list','real_coords']).cache()
#avgHB = dataframe.rdd.map(lambda x: int(x["hb_length"])).reduce(lambda x,y:x+y)/dataframe.count()
#std = dataframe.rdd.map(lambda x: int(x["hb_length"])).stdev()

print("%s" % (avgHB))

Deleted 677project/output/sparkoutput/dump.ch2oh33_prod.lammpstrj
1.4231519059662432
CPU times: user 284 ms, sys: 72.5 ms, total: 356 ms
Wall time: 16min 22s


In [35]:
!hdfs dfs -rm -r -skipTrash 677project/output/sparkoutput/dump.ch2oh1212_prod.lammpstrj

Deleted 677project/output/sparkoutput/dump.ch2oh1212_prod.lammpstrj


In [6]:
!hdfs dfs -du -h 677project/output/sparkoutput

259.1 M  677project/output/sparkoutput/ch2oh1212_prod
842.1 M  677project/output/sparkoutput/ch2oh33_prod
144.4 M  677project/output/sparkoutput/ch2oh99_prod
190.8 K  677project/output/sparkoutput/ch2oh_dump_1ns.lammpstrj
259.1 M  677project/output/sparkoutput/dump.ch2oh1212_prod.lammpstrj
842.1 M  677project/output/sparkoutput/dump.ch2oh33_prod.lammpstrj
70.7 M   677project/output/sparkoutput/dump.ch2oh66_prod.lammpstrj
144.4 M  677project/output/sparkoutput/dump.ch2oh99_prod.lammpstrj
190.8 K  677project/output/sparkoutput/oh_1ns.lammpstrj


In [10]:
!hdfs dfs -copyToLocal 677project/data/dump.ch2oh1212_prod.lammpstrj /scratch2/yli25

In [4]:
!ls /scratch1/xiaohoz/for_YuLi

ch2oh.10g.lammpstrj  ch2oh.1g.lammpstrj


In [7]:
!cp /scratch1/xiaohoz/for_YuLi/ch2oh.1g.lammpstrj /scratch2/yli25

In [1]:
!ls -sh /scratch2/yli25

total 47G
9.7G ch2oh.10g.lammpstrj	     28G dump.ch2oh33_prod.lammpstrj
968M ch2oh.1g.lammpstrj		     45K __jupyterhub__
8.6G dump.ch2oh1212_prod.lammpstrj


In [1]:
!ls -sh ./data/ch2oh_dump_1ns.lammpstrj

6.3M ./data/ch2oh_dump_1ns.lammpstrj


In [15]:
!wc -l /scratch2/yli25/dump.ch2oh33_prod.lammpstrj

913500203 /scratch2/yli25/dump.ch2oh33_prod.lammpstrj


In [ ]:
968M
1.2325732484076433
CPU times: user 147 ms, sys: 43.9 ms, total: 191 ms
Wall time: 4min 59s
9.7G
1.4266803559206023
CPU times: user 206 ms, sys: 45.5 ms, total: 252 ms
Wall time: 7min 14s
28G
1.4231519059662432
CPU times: user 284 ms, sys: 72.5 ms, total: 356 ms
Wall time: 16min 22s

In [12]:
!hdfs dfs -ls -h 677project/data

Found 6 items
-rw-r--r--   2 yli25 hdfs-user       9.7 G 2018-03-18 23:51 677project/data/ch2oh.10g.lammpstrj
-rw-r--r--   2 yli25 hdfs-user     972.6 M 2018-03-18 23:46 677project/data/ch2oh.1g.lammpstrj
-rw-r--r--   2 yli25 hdfs-user       6.2 M 2017-11-15 15:26 677project/data/ch2oh_dump_1ns.lammpstrj
-rw-r--r--   2 yli25 hdfs-user      63.5 K 2017-11-29 15:34 677project/data/ch2oh_short.lammpstrj
-rw-r--r--   2 yli25 hdfs-user      27.3 G 2017-11-24 22:58 677project/data/dump.ch2oh33_prod.lammpstrj
-rw-r--r--   2 yli25 hdfs-user       4.7 M 2017-11-06 16:29 677project/data/dump.co.lammpstrj


In [12]:
!hdfs dfs -cat 677project/output/sparkoutput/oh_1ns.lammpstrj/part-00000 \
    2>/dev/null | head -n 20

(0, 2, [(76, 178, 179), (39, 104, 105)], {178: (0.306188, 0.953484, 0.233001), 179: (0.398668, 0.972962, 0.268367), 39: (0.981067, 0.356205, 0.229215), 104: (0.878979, 0.423503, 0.212988), 105: (0.897277, 0.409454, 0.250986), 76: (0.425822, 0.942096, 0.240397)})
(1000, 1, [(76, 178, 179)], {178: (0.451192, 1.0727878, 0.237461), 179: (0.561458, 0.9712479, 0.272913), 76: (0.560791, 0.961417, 0.2445)})
(2000, 2, [(76, 178, 179), (39, 104, 105)], {178: (0.342672, 1.00525, 0.266657), 179: (0.538719, 0.898728, 0.278641), 39: (0.9603, 0.275051, 0.228717), 104: (0.843395, 0.384878, 0.22668), 105: (1.0398646, 0.302524, 0.214019), 76: (0.455294, 0.892184, 0.261285)})
(3000, 2, [(76, 178, 179), (39, 104, 105)], {178: (0.420508, 1.00257108, 0.255065), 179: (0.609856, 0.907056, 0.269455), 39: (1.00742846, 0.265535, 0.204173), 104: (0.895508, 0.381014, 0.20634), 105: (1.0426105, 0.27094, 0.178388), 76: (0.537972, 0.892895, 0.250764)})
(4000, 2, [(76, 178, 179), (39, 104, 105)], {178: (0.409058, 1.03

In [19]:
sc.stop()